# Amazon Chronos

In [2]:
from chronos import BaseChronosPipeline
import torch
import pandas as pd
import numpy as np
from methodology.forecasting import compute_rmse_df

/Users/henry/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Load the data and calculate the inflation rate
cpi_monthly_data = pd.read_csv("methodology/data/cpi_monthly_data.csv", index_col=0)
cpi = cpi_monthly_data[["00"]]
inflation_rate = cpi.pct_change(12).dropna(how="all") * 100
inflation_rate.columns = ["Inflation Rate"]
inflation_rate.index = pd.to_datetime(inflation_rate.index)

In [4]:
dates = inflation_rate.loc["2009-12-01":"2024-11-01"].index
models = ["Chronos (Tiny)", "Chronos (Base)", "Chronos (Large)"]
chronos_df = pd.DataFrame(index=dates, columns=pd.MultiIndex.from_product([models, range(1, 13)]))

In [5]:
# Chronos (Tiny)
pipeline = BaseChronosPipeline.from_pretrained(
    "amazon/chronos-t5-tiny", 
    device_map="cpu",
    torch_dtype=torch.bfloat16,
)

for date in dates:
    # Get the inflation rate data up to the current time step
    past_data = inflation_rate.loc[:date].to_numpy().flatten()
    # Make a forecast for the next 12 months
    quantiles, mean = pipeline.predict_quantiles(
        context=torch.tensor(past_data),
        prediction_length=12,
        quantile_levels=[0.1, 0.5, 0.9],
    )
    chronos_df.loc[date, "Chronos (Tiny)"] = mean.flatten().numpy()

In [9]:
# Chronos (Base)
pipeline = BaseChronosPipeline.from_pretrained(
    "amazon/chronos-t5-base", 
    device_map="cpu",
    torch_dtype=torch.bfloat16,
)

for date in dates:
    # Get the inflation rate data up to the current time step
    past_data = inflation_rate.loc[:date].to_numpy().flatten()
    # Make a forecast for the next 12 months
    quantiles, mean = pipeline.predict_quantiles(
        context=torch.tensor(past_data),
        prediction_length=12,
        quantile_levels=[0.1, 0.5, 0.9],
    )
    chronos_df.loc[date, "Chronos (Base)"] = mean.flatten().numpy()

In [13]:
# Chronos (Large)
pipeline = BaseChronosPipeline.from_pretrained(
    "amazon/chronos-t5-large", 
    device_map="cpu",
    torch_dtype=torch.bfloat16,
)

for date in dates:
    # Get the inflation rate data up to the current time step
    past_data = inflation_rate.loc[:date].to_numpy().flatten()
    # Make a forecast for the next 12 months
    quantiles, mean = pipeline.predict_quantiles(
        context=torch.tensor(past_data),
        prediction_length=12,
        quantile_levels=[0.1, 0.5, 0.9],
    )
    chronos_df.loc[date, "Chronos (Large)"] = mean.flatten().numpy()

In [17]:
np.round(compute_rmse_df(chronos_df).T, 2)

,1,2,3,4,5,6,7,8,9,10,11,12
Chronos (Tiny),0.41,0.63,0.84,1.05,1.29,1.53,1.76,2.00,2.22,2.42,2.62,2.82
Chronos (Base),0.43,0.66,0.84,1.06,1.29,1.50,1.72,1.95,2.15,2.32,2.50,2.68
Chronos (Large),0.43,0.65,0.83,1.04,1.27,1.47,1.69,1.90,2.10,2.26,2.43,2.59


In [15]:
chronos_df.to_csv("results/benchmarks/chronos_forecasts.csv")